In [173]:
path = './zomato.csv'

In [174]:
import csv
import os
import glob
import numpy as np
import math
from sklearn.metrics import accuracy_score

In [175]:
def cleaning(sentence):
    sentence = sentence.replace('\ufeff','')
    sentence = sentence.replace('\n',' ')
    sentence = sentence.replace('\r',' ')
    sentence = sentence.replace('\n\n',' ')
    sentence = sentence.replace('\r\r',' ')
    sentence = sentence.replace('.',' ')
    sentence = sentence.replace(',',' ')
    sentence = sentence.replace('"',' ')
    sentence = sentence.replace('!',' ')
    sentence = sentence.replace('?',' ')
    sentence = sentence.replace('/',' ')
    sentence = sentence.replace('-',' ')
    sentence = sentence.lstrip()
    return sentence.lower()


def count_word_in_class(x,y):
    count = {}
    count_word = {}
    word_class = {}
    for i in range(0, len(x)):
        x_split = x[i].split()
        for word in x_split:
            w_c = word+str(",")+str(y[i])
            if (w_c in count):
                count[w_c] += 1
            else:
                count[w_c] = 1
                
            if (word in count_word):
                count_word[word] += 1
            else:
                count_word[word] = 1
                
            if (y[i] in word_class):
                word_class[y[i]] += 1
            else:
                word_class[y[i]] = 1
    return count, count_word, word_class

def count_class_in_doc(y):
    count = {}
    for i in range(0, len(y)):
        if (y[i] in count):
            count[y[i]] += 1
        else:
            count[y[i]] = 1
    return count

def hitung_prob(count,word,uniq):
    return ((count + 1)/(word+uniq))

In [176]:
x = []
y = []
with open(path) as a:
    b = csv.reader(a)
    for row in b:
        x.append(cleaning(row[2]))
        y.append(row[3])

In [177]:
randomize = np.arange(len(x))
np.random.shuffle(randomize)

In [183]:
x_train = x[:85]
y_train = y[:85]

x_test = x[85:]
y_test = y[85:]

In [184]:
count_word_on_class, count_word, word_class = count_word_in_class(x_train,y_train)

In [185]:
count_class = count_class_in_doc(y_train)

In [186]:
kelaz = list(count_class.keys())
y_pred = []
for i in range(0,len(x_test)):
    prob_kelas = {}
    for j in count_class.keys():
        prob_c = 1
        x_split = x_test[i].split()
        for k in x_split:
            tag = k+str(',')+str(j)
            prob_c += math.log(hitung_prob(count_word_on_class.get(tag,0),word_class.get(j),len(count_word.keys())))
        nama = str(i)+','+str(j)
        prob_kelas[nama] = prob_c
    y_pred.append(kelaz[np.argmax(list(prob_kelas.values()))])
    

In [187]:
accuracy_score(y_test, y_pred)

0.7333333333333333

In [188]:
# y_test

In [189]:
# y_pred

### Coba KFold Validation

In [190]:
from sklearn.model_selection import KFold

x = []
y = []
x_train = []
y_train = []
x_test = []
y_test = []
with open(path) as a:
    b = csv.reader(a)
    for row in b:
        x.append(cleaning(row[2]))
        y.append(row[3])
x = np.array(x)
y = np.array(y)
        
randomize = np.arange(len(x))
np.random.shuffle(randomize)

acc = []
folds = 6
kf = KFold(n_splits=folds)
for train, test in kf.split(x):
    x_train, x_test, y_train, y_test = x[train],x[test],y[train],y[test]
    
    count_word_on_class, count_word, word_class = count_word_in_class(x_train,y_train)
    count_class = count_class_in_doc(y_train)
    kelaz = list(count_class.keys())
    y_pred = []
    for i in range(0,len(x_test)):
        prob_kelas = {}
        for j in count_class.keys():
            prob_c = 1
            x_split = x_test[i].split()
            for k in x_split:
                tag = k+str(',')+str(j)
                prob_c += math.log(hitung_prob(count_word_on_class.get(tag,0),word_class.get(j),len(count_word.keys())))
            nama = str(i)+','+str(j)
            prob_kelas[nama] = prob_c
        y_pred.append(kelaz[np.argmax(list(prob_kelas.values()))])
    acc.append(accuracy_score(y_test, y_pred))
print("Mean Acc of",folds,"Fold:",np.mean(acc))

Mean Acc of 6 Fold: 0.7212009803921569
